Part 1

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/Users/houhiroshisakai/Desktop/Schulich/Term 2/MBAN 5110/midterm_partone.csv')

In [4]:
df

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446
...,...,...,...,...,...,...,...,...
1691,1,-0.015543,5.225766,0.309119,1.615384,3.554503,2.197871,0.005549
1692,1,0.399089,5.324390,0.274782,1.463044,3.745006,2.324502,0.004359
1693,1,-0.702200,5.575258,0.287503,1.602905,3.434909,2.282626,0.000000
1694,1,0.283926,5.423463,0.256657,1.391968,2.876645,1.454948,0.000000


In [5]:
import numpy as np
from statsmodels.sandbox.regression.gmm import GMM

y = np.array(df['Stock Change'])
x = np.array(df[['Inventory Turnover','Operating Profit','Interaction Effect']])
z = np.array(df[['Current Ratio','Quick Ratio','Debt Asset Ratio']])

In [6]:
class gmm(GMM):
    def momcond(self, params):
        p0,p1,p2,p3,delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument
        error0 = (endog - p0 - p1 *  exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2])
        error1 = error0 * exog[:,1]
        error2 = error0 * exog[:,2]
        error3 = (error0 - delta) * inst[:,0] 
        error4 = (error0 - delta) * inst[:,1]
        error5 = (error0 - delta) * inst[:,2]

        g = np.column_stack((error0,error1,error2,error3,error4,error5))
        return g
    
beta0 = np.array([0.1,0.1,0.1,0.1,1])
res = gmm(endog = y, exog = x, instrument = z, k_moms= 6, k_params= 4).fit(beta0)
res.summary()

Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 13
         Function evaluations: 17
         Gradient evaluations: 17
Optimization terminated successfully.
         Current function value: 0.000205
         Iterations: 9
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: 0.000205
         Iterations: 4
         Function evaluations: 8
         Gradient evaluations: 8
Optimization terminated successfully.
         Current function value: 0.000205
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.3475
Model:                            gmm   Prob (Hansen J):                 0.556
Method:                           GMM                                         
Date:                Sat, 11 Nov 2023                                         
Time:                        13:41:06                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0139      0.024     -0.588      0.556      -0.060       0.033
p 1            0.0009      0.001      1.217      0.224      -0.001       0.002
p 2           -0.1103      0.032     -3.416      0.001      -0.174      -0.047
p 3            0.0012      0.000      2.782      0.005       0.000       0.002
p 4           -0.0048      0.009     -0.534      0.594      -0.022       0.013
==============================================================================
"""

In [7]:
model = gmm(endog = y, exog = x, instrument=z, k_moms=6, k_params=4)
res = model.fit(beta0)

params = res.params
for i in range(1,10):
    moms = model.momcond(params)
    weights = model.calc_weightmatrix(moms)
    model.w = weights
    res = model.fit(start_params = params, maxiter = 1, wargs = {'centered': False})
    params = res.params

res.summary()


Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 13
         Function evaluations: 17
         Gradient evaluations: 17
Optimization terminated successfully.
         Current function value: 0.000205
         Iterations: 9
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: 0.000205
         Iterations: 4
         Function evaluations: 8
         Gradient evaluations: 8
Optimization terminated successfully.
         Current function value: 0.000205
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5
Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 15
         Function evaluations: 19
         Gradient evaluations: 19
Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 0
         Function evaluations: 1
         Gr

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                     0.009153
Model:                            gmm   Prob (Hansen J):                 0.924
Method:                           GMM                                         
Date:                Sat, 11 Nov 2023                                         
Time:                        13:41:06                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0077      0.024     -0.323      0.747      -0.054       0.039
p 1            0.0006      0.001      0.884      0.377      -0.001       0.002
p 2           -0.1150      0.032     -3.567      0.000      -0.178      -0.052
p 3            0.0013      0.000      2.989      0.003       0.000       0.002
p 4           -0.0101      0.009     -1.125      0.261      -0.028       0.008
==============================================================================
"""

Part 2

In [8]:
df_pt2 = pd.read_csv('/Users/houhiroshisakai/Desktop/Schulich/Term 2/MBAN 5110/midterm_parttwo.csv')

In [9]:
df_pt2

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative
...,...,...,...,...,...,...,...
8076,3,Low,Less than 2,Very High,Very high,Married,Positive
8077,3,Medium,Less than 2,Very High,Very high,Married,Negative
8078,3,Medium,More than 2,Very High,Very high,Married,Positive
8079,7,Medium,Less than 2,Very High,Very high,Married,Positive


In [10]:
df_pt2 = pd.get_dummies(df_pt2, columns=['Requested Credit Amount','Number of Dependents','Monthly Income','Monthly Expense','Marital Status','Credit Rating'],drop_first = True)

In [11]:
df_pt2

,Years of Education after High School,Requested Credit Amount_Low,Requested Credit Amount_Medium,Number of Dependents_More than 2,Number of Dependents_No dependent,Monthly Income_Low,Monthly Income_Moderate,Monthly Income_Very High,Monthly Income_Very low,Monthly Expense_Low,Monthly Expense_Moderate,Monthly Expense_Very high,Monthly Expense_Very low,Marital Status_Not specified,Marital Status_Single,Credit Rating_Positive
0,1,True,False,False,True,False,False,False,True,False,False,False,True,False,False,True
1,2,True,False,False,True,False,False,False,True,False,False,False,True,False,True,True
2,1,True,False,False,True,False,False,False,True,False,False,False,True,False,True,True
3,3,True,False,False,True,False,False,False,True,False,False,False,True,False,False,True
4,3,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8076,3,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True
8077,3,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False
8078,3,False,True,True,False,False,False,True,False,False,False,True,False,False,False,True
8079,7,False,True,False,False,False,False,True,False,False,False,True,False,False,False,True


In [12]:
from sklearn.model_selection import train_test_split

y = df_pt2['Credit Rating_Positive']
x = df_pt2.drop('Credit Rating_Positive',axis = 1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5, random_state=42)

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [14]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score

y_pred = model.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test,y_pred)

print('Confusion Matrix is: \n', cm)
print('Recall: ', recall)
print('Precision: ', precision)
print('F1 Score: ', f1)

Confusion Matrix is: 
 [[   0  577]
 [   0 3464]]
Recall:  1.0
Precision:  0.8572135609997525
F1 Score:  0.9231179213857428


In [15]:
y_prob = model.predict_proba(x_test)[:,1]

threshold = np.percentile(y_prob, 85)

y_pred_adjusted = (y_prob > threshold).astype(int)

cm_adjusted = confusion_matrix(y_test, y_pred_adjusted)
recall_adjusted = recall_score(y_test, y_pred_adjusted)
precision_adjusted = precision_score(y_test, y_pred_adjusted)
f1_adjusted = f1_score(y_test,y_pred_adjusted)

print('Adjusted Confustion Matrix is: \n', cm_adjusted)
print('Adjusted Recall: ', recall_adjusted)
print('Adjusted Precision: ', precision_adjusted)
print('Adjusted F1 Score: ', f1_adjusted)

Adjusted Confustion Matrix is: 
 [[ 496   81]
 [2940  524]]
Adjusted Recall:  0.151270207852194
Adjusted Precision:  0.8661157024793389
Adjusted F1 Score:  0.2575571393462768
